In [54]:
# import important libraries

import pandas as pd
import numpy as np
import random
from itertools import combinations

import gurobipy as gp
from gurobipy import GRB

In [55]:
# import data from excel file
# fill null values(from city i to i) with a very large number
# delete unnecessary columns
# set a list of cities to use for distance list 
# set the cities column as new index

data = pd.read_excel("Desktop/TR81_KGM.xls")
data.fillna(value = 1000000000, inplace = True)
data = data.drop(['İL NO'], axis=1)
data = data.drop(['İL ADI.1'], axis=1)
data = data.drop(['İL NO.1'], axis=1)
cities = data['İL ADI'].values.tolist()
data = data.set_index('İL ADI')
data.head(10)

,Adana,Adıyaman,Afyonkarahisar,Ağrı,Amasya,Ankara,Antalya,Artvin,Aydın,Balıkesir,...,Batman,Şırnak,Bartın,Ardahan,Iğdır,Yalova,Karabük,Kilis,Osmaniye,Düzce
İL ADI,,,,,,,,,,,,,,,,,,,,,
Adana,1.000000e+09,3.330000e+02,5.730000e+02,9.650000e+02,6.110000e+02,4.900000e+02,5.580000e+02,1.034000e+03,8.830000e+02,8.970000e+02,...,618.0,717.0,769.0,1038.0,1068.0,893.0,701.0,246.0,87.0,722.0
Adıyaman,3.330000e+02,1.000000e+09,9.060000e+02,6.480000e+02,6.340000e+02,7.550000e+02,8.910000e+02,7.550000e+02,1.216000e+03,1.230000e+03,...,301.0,480.0,1038.0,759.0,751.0,1162.0,970.0,209.0,246.0,991.0
Afyonkarahisar,5.730000e+02,9.060000e+02,1.000000e+09,1.311000e+03,5.900000e+02,2.560000e+02,2.920000e+02,1.237000e+03,3.460000e+02,3.240000e+02,...,1191.0,1290.0,515.0,1346.0,1421.0,341.0,447.0,819.0,660.0,375.0
Ağrı,9.650000e+02,6.480000e+02,1.311000e+03,1.000000e+09,7.380000e+02,1.055000e+03,1.432000e+03,3.970000e+02,1.645000e+03,1.571000e+03,...,369.0,426.0,1173.0,310.0,143.0,1363.0,1104.0,815.0,878.0,1192.0
Amasya,6.110000e+02,6.340000e+02,5.900000e+02,7.380000e+02,1.000000e+09,3.340000e+02,8.260000e+02,6.950000e+02,9.320000e+02,8.330000e+02,...,800.0,982.0,435.0,779.0,848.0,625.0,366.0,666.0,629.0,454.0
Ankara,4.900000e+02,7.550000e+02,2.560000e+02,1.055000e+03,3.340000e+02,1.000000e+09,5.440000e+02,9.810000e+02,5.980000e+02,5.330000e+02,...,1010.0,1179.0,283.0,1090.0,1165.0,407.0,215.0,730.0,577.0,236.0
Antalya,5.580000e+02,8.910000e+02,2.920000e+02,1.432000e+03,8.260000e+02,5.440000e+02,1.000000e+09,1.469000e+03,3.420000e+02,5.050000e+02,...,1176.0,1275.0,803.0,1473.0,1542.0,605.0,735.0,804.0,645.0,639.0
Artvin,1.034000e+03,7.550000e+02,1.237000e+03,3.970000e+02,6.950000e+02,9.810000e+02,1.469000e+03,1.000000e+09,1.579000e+03,1.463000e+03,...,603.0,754.0,1048.0,117.0,344.0,1255.0,979.0,922.0,947.0,1084.0
Aydın,8.830000e+02,1.216000e+03,3.460000e+02,1.645000e+03,9.320000e+02,5.980000e+02,3.420000e+02,1.579000e+03,1.000000e+09,2.960000e+02,...,1501.0,1600.0,857.0,1686.0,1755.0,510.0,789.0,1129.0,970.0,673.0


In [56]:
# making a list with all cities' combinations of 2 and the distances between them

dist = {(c1, c2): data.loc[c1, c2] for c1, c2 in combinations(cities, 2)}
dist

{('Adana', 'Adıyaman'): 333.0,
 ('Adana', 'Afyonkarahisar'): 573.0,
 ('Adana', 'Ağrı'): 965.0,
 ('Adana', 'Amasya'): 611.0,
 ('Adana', 'Ankara'): 490.0,
 ('Adana', 'Antalya'): 558.0,
 ('Adana', 'Artvin'): 1034.0,
 ('Adana', 'Aydın'): 883.0,
 ('Adana', 'Balıkesir'): 897.0,
 ('Adana', 'Bilecik'): 768.0,
 ('Adana', 'Bingöl'): 632.0,
 ('Adana', 'Bitlis'): 731.0,
 ('Adana', 'Bolu'): 677.0,
 ('Adana', 'Burdur'): 666.0,
 ('Adana', 'Bursa'): 837.0,
 ('Adana', 'Çanakkale'): 1097.0,
 ('Adana', 'Çankırı'): 576.0,
 ('Adana', 'Çorum'): 579.0,
 ('Adana', 'Denizli'): 760.0,
 ('Adana', 'Diyarbakır'): 522.0,
 ('Adana', 'Edirne'): 1169.0,
 ('Adana', 'Elâzığ'): 490.0,
 ('Adana', 'Erzincan'): 677.0,
 ('Adana', 'Erzurum'): 808.0,
 ('Adana', 'Eskişehir'): 686.0,
 ('Adana', 'Gaziantep'): 209.0,
 ('Adana', 'Giresun'): 727.0,
 ('Adana', 'Gümüşhane'): 786.0,
 ('Adana', 'Hakkâri'): 906.0,
 ('Adana', 'Hatay'): 191.0,
 ('Adana', 'Isparta'): 616.0,
 ('Adana', 'Mersin'): 69.0,
 ('Adana', 'İstanbul'): 939.0,
 ('Adana

In [57]:
#Creating Model

m = gp.Model()

# Adding variable xi,j
vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')

# Symmetry Constraints (2)
for i, j in vars.keys():
    vars[j, i] = vars[i, j]

# Entering and leaving a city constraints (3):
cons = m.addConstrs(vars.sum(c, '*') == 2 for c in cities)

In [58]:
# Lazy constraints to eliminate sub-tours (4)
# function finds the shortest subtour and then adds that subtour as a constraint to subtour elimination

def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)
        tour = subtour(selected)
        if len(tour) < len(cities):
            # add subtour elimination constraint for every pair of cities in subtour
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2)) <= len(tour)-1)

            
# To find the shortest subtour

def subtour(edges):
    unvisited = cities[:]
    cycle = cities[:] 
    while unvisited:  
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # cycle: New shortest subtour
    return cycle

In [59]:
# adding variables and constraints and running the model

m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 81 rows, 3240 columns and 6480 nonzeros
Model fingerprint: 0xbfe20205
Variable types: 0 continuous, 3240 integer (3240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 81 rows, 3240 columns, 6480 nonzeros
Variable types: 0 continuous, 3240 integer (3240 binary)

Root relaxation: objective 9.562000e+03, 115 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9562.00000    0    6          - 9562.00000      -     -    0s
     0     0 9782.00000    0    8          

In [60]:
# verifying that the optimal tour goes to all the cities and returns to the origin city

vals = m.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == len(cities)

In [61]:
# Read capital names and coordinates from json file
# Create a coordinates list for the map

import json

try:
  cities_of_turkey_json = json.load(open('cities_of_turkey.json'))
except:
  import urllib.request
  url = 'https://gist.githubusercontent.com/ozdemirburak/4821a26db048cc0972c1beee48a408de/raw/4754e5f9d09dade2e6c461d7e960e13ef38eaa88/cities_of_turkey.json'
  data = urllib.request.urlopen(url).read()
  cities_of_turkey_json = json.loads(data)

name = []
coordinates = {}
for i in cities_of_turkey_json:
      city = i['name']
      cities.append(city)
      coordinates[city] = (float(i['latitude']), float(i['longitude']))

In [62]:
# Map the solution

import folium

map = folium.Map(location=[40,35], zoom_start = 6)

points = []
for city in tour:
  points.append(coordinates[city])
points.append(points[0])

folium.PolyLine(points).add_to(map)

map

In [63]:
# list of the visited cities in order 

tour

['Adana',
 'Mersin',
 'Karaman',
 'Konya',
 'Aksaray',
 'Nevşehir',
 'Niğde',
 'Kayseri',
 'Sivas',
 'Tokat',
 'Amasya',
 'Çorum',
 'Yozgat',
 'Kırşehir',
 'Kırıkkale',
 'Çankırı',
 'Ankara',
 'Eskişehir',
 'Kütahya',
 'Afyonkarahisar',
 'Uşak',
 'Isparta',
 'Burdur',
 'Antalya',
 'Denizli',
 'Muğla',
 'Aydın',
 'İzmir',
 'Manisa',
 'Balıkesir',
 'Çanakkale',
 'Edirne',
 'Kırklareli',
 'Tekirdağ',
 'İstanbul',
 'Kocaeli',
 'Yalova',
 'Bursa',
 'Bilecik',
 'Sakarya',
 'Bolu',
 'Düzce',
 'Zonguldak',
 'Bartın',
 'Karabük',
 'Kastamonu',
 'Sinop',
 'Samsun',
 'Ordu',
 'Giresun',
 'Trabzon',
 'Rize',
 'Artvin',
 'Ardahan',
 'Kars',
 'Iğdır',
 'Ağrı',
 'Erzurum',
 'Bayburt',
 'Gümüşhane',
 'Erzincan',
 'Tunceli',
 'Malatya',
 'Elâzığ',
 'Bingöl',
 'Muş',
 'Bitlis',
 'Van',
 'Hakkâri',
 'Şırnak',
 'Siirt',
 'Batman',
 'Diyarbakır',
 'Mardin',
 'Şanlıurfa',
 'Adıyaman',
 'Kahramanmaraş',
 'Gaziantep',
 'Kilis',
 'Hatay',
 'Osmaniye']